- ref:https://www.kaggle.com/kaerunantoka/h-m-eda-w-pyspark

In [1]:
!pip install pyspark > /dev/null

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('h-and-m-personalized-fashion-recommendations').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/23 08:58:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
class CFG:
    TRANSACTION_PATH = '../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
    ARTICLE_PATH = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
    CUSTOMER_PATH = '../input/h-and-m-personalized-fashion-recommendations/customers.csv'
    SAMPLE_PATH = '../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'
    IMAGE_PATH = '../input/h-and-m-personalized-fashion-recommendations/images'

# log data

In [4]:
train = spark.read.option('header','true').csv(CFG.TRANSACTION_PATH)

In [5]:
train.count()

31788324

In [6]:
train.show(5, False)

+----------+----------------------------------------------------------------+----------+--------------------+----------------+
|t_dat     |customer_id                                                     |article_id|price               |sales_channel_id|
+----------+----------------------------------------------------------------+----------+--------------------+----------------+
|2018-09-20|000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318|0663713001|0.050830508474576264|2               |
|2018-09-20|000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318|0541518023|0.03049152542372881 |2               |
|2018-09-20|00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2|0505221004|0.01523728813559322 |2               |
|2018-09-20|00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2|0685687003|0.016932203389830508|2               |
|2018-09-20|00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2|0685687004|0.016932203389830508|2 